# Полноценный tutorial по Policyscope

В этом ноутбуке показано, как на синтетических данных сравнить две рекомендательные политики без онлайн A/B‑теста. Мы:
1. сгенерируем пользователей и логи политики A с корректными пропенсити;
2. оценим политику B методами Replay, IPS, SNIPS, DM и Doubly Robust;
3. построим доверительные интервалы через бутстрэп;
4. сравним оффлайн‑оценки с истинным эффектом (оракулом).

## 1. Генерация данных

Среда имитирует выдачу банковских предложений. Политика A стохастическая (ε‑greedy), поэтому в логах есть вероятность выбранного действия.

In [1]:

from policyscope.synthetic import SynthConfig, SyntheticRecommenderEnv
from policyscope.policies import make_policy
from policyscope.estimators import (
    train_mu_hat, prepare_piB_taken, replay_value, ips_value, snips_value, dm_value, dr_value
)
from policyscope.bootstrap import paired_bootstrap_ci
import pandas as pd

cfg = SynthConfig(n_users=2000, seed=0)
env = SyntheticRecommenderEnv(cfg)
users = env.sample_users()
users.head()


,user_id,loyal,age,risk,income,region,age_z,risk_z,income_z
0,0,0,39,0.164427,33847.243699,2,-0.083333,-1.342291,-0.151654
1,1,1,68,0.794970,28464.133287,1,2.333333,1.179882,-0.496750
2,2,1,51,0.173588,106405.425932,1,0.916667,-1.305647,2.130351
3,3,1,21,0.460746,23807.909876,0,-1.583333,-0.157015,-0.852632
4,4,0,58,0.129342,22354.664095,2,1.500000,-1.482632,-0.978114


## 2. Политики A и B

- **A**: ε‑greedy (ε=0.15) на скрытых скоринговых функциях.
- **B**: softmax (τ=0.7) на тех же скорингах.

Сгенерируем логи A и посмотрим первые строки.

In [2]:

policyA = make_policy('epsilon_greedy', seed=1, epsilon=0.15)
policyB = make_policy('softmax', seed=2, tau=0.7)
logsA = env.simulate_logs_A(policyA, users)
logsA.head()


,user_id,loyal,age,risk,income,region,age_z,risk_z,income_z,a_A,propensity_A,accept,cltv
0,0,0,39,0.164427,33847.243699,2,-0.083333,-1.342291,-0.151654,0,0.8875,1,534.149918
1,1,1,68,0.794970,28464.133287,1,2.333333,1.179882,-0.496750,1,0.8875,0,605.271806
2,2,1,51,0.173588,106405.425932,1,0.916667,-1.305647,2.130351,0,0.8875,1,1552.634426
3,3,1,21,0.460746,23807.909876,0,-1.583333,-0.157015,-0.852632,2,0.8875,1,751.644903
4,4,0,58,0.129342,22354.664095,2,1.500000,-1.482632,-0.978114,0,0.8875,1,470.883121


## 3. Оценка value(A) и value(B)

Метрики:
- **accept** — отклик пользователя (0/1);
- **cltv** — изменённая ценность за горизонт 90 дней.

Рассчитаем V(A) и оценки V(B) разными методами.

In [3]:

vA_accept = logsA['accept'].mean()
vA_cltv = logsA['cltv'].mean()

piB_taken = prepare_piB_taken(logsA, policyB)
mu_accept = train_mu_hat(logsA, target='accept')
mu_cltv = train_mu_hat(logsA, target='cltv')

res = []
# Replay
res.append(('Replay',
             replay_value(logsA, policyB.action_argmax(users), 'accept'),
             replay_value(logsA, policyB.action_argmax(users), 'cltv')))
# IPS
res_ip_accept, ess_ip_a, clip_ip_a = ips_value(logsA, piB_taken, 'accept', weight_clip=20)
res_ip_cltv, ess_ip_c, clip_ip_c = ips_value(logsA, piB_taken, 'cltv', weight_clip=20)
res.append(('IPS', res_ip_accept, res_ip_cltv))
# SNIPS
res_sn_accept, ess_sn_a, clip_sn_a = snips_value(logsA, piB_taken, 'accept', weight_clip=20)
res_sn_cltv, ess_sn_c, clip_sn_c = snips_value(logsA, piB_taken, 'cltv', weight_clip=20)
res.append(('SNIPS', res_sn_accept, res_sn_cltv))
# DM
res.append(('DM', dm_value(logsA, policyB, mu_accept, 'accept'), dm_value(logsA, policyB, mu_cltv, 'cltv')))
# DR
res_dr_accept, ess_dr_a, clip_dr_a = dr_value(logsA, policyB, mu_accept, 'accept', weight_clip=20)
res_dr_cltv, ess_dr_c, clip_dr_c = dr_value(logsA, policyB, mu_cltv, 'cltv', weight_clip=20)
res.append(('DR', res_dr_accept, res_dr_cltv))

pd.DataFrame(res, columns=['method','V_B_accept','V_B_cltv'])


,method,V_B_accept,V_B_cltv
0,Replay,0.522857,716.402723
1,IPS,0.441777,612.060087
2,SNIPS,0.478821,663.382246
3,DM,0.491499,679.777741
4,DR,0.481470,674.928823


## 4. Доверительные интервалы для DR

Используем кластерный бутстрэп по пользователям, чтобы получить 95% доверительные интервалы для разности ценностей (ATE).

In [4]:

def estimator_pair_accept(df):
    mu = train_mu_hat(df, 'accept')
    vA = df['accept'].mean()
    vB, _, _ = dr_value(df, policyB, mu, 'accept', weight_clip=20)
    return vA, vB, vB - vA

def estimator_pair_cltv(df):
    mu = train_mu_hat(df, 'cltv')
    vA = df['cltv'].mean()
    vB, _, _ = dr_value(df, policyB, mu, 'cltv', weight_clip=20)
    return vA, vB, vB - vA

ci_accept = paired_bootstrap_ci(logsA, estimator_pair_accept)
ci_cltv = paired_bootstrap_ci(logsA, estimator_pair_cltv)
ci_accept, ci_cltv


({'V_A': np.float64(0.5135),
  'V_A_CI': (0.4986213279153681, 0.5293527670743284),
  'V_B': 0.48146966174506906,
  'V_B_CI': (0.4459537202535492, 0.5171078060588417),
  'Delta': np.float64(-0.0320303382549309),
  'Delta_CI': (-0.06477892996017516, 0.00018136549190642356),
  'n_boot': 300},
 {'V_A': np.float64(687.670207586939),
  'V_A_CI': (674.8326503921056, 698.6395015882501),
  'V_B': 674.928823485076,
  'V_B_CI': (660.0687294091315, 690.4445440723413),
  'Delta': np.float64(-12.741384101862991),
  'Delta_CI': (-23.18077557141469, -1.5732426661529675),
  'n_boot': 300})

## 5. Сравнение с "Оракулом"

Так как мы знаем генеративную модель, можно вычислить истинные ожидания V(A) и V(B) и проверить точность OPE.

In [5]:

oracle_A_accept = env.oracle_value(policyA, users, metric='accept')
oracle_B_accept = env.oracle_value(policyB, users, metric='accept')
oracle_A_cltv = env.oracle_value(policyA, users, metric='cltv')
oracle_B_cltv = env.oracle_value(policyB, users, metric='cltv')

oracle_df = pd.DataFrame({
    'metric':['accept','cltv'],
    'V_A_true':[oracle_A_accept, oracle_A_cltv],
    'V_B_true':[oracle_B_accept, oracle_B_cltv],
    'Delta_true':[oracle_B_accept - oracle_A_accept, oracle_B_cltv - oracle_A_cltv],
    'DR_estimate':[res_dr_accept, res_dr_cltv],
    'abs_error':[abs(res_dr_accept-(oracle_B_accept - oracle_A_accept)),
                 abs(res_dr_cltv-(oracle_B_cltv - oracle_A_cltv))]
})
oracle_df


,metric,V_A_true,V_B_true,Delta_true,DR_estimate,abs_error
0,accept,0.54050,0.514500,-0.026000,0.481470,0.507470
1,cltv,696.39775,680.141577,-16.256173,674.928823,691.184997


## 6. Вывод

- Политика **B** улучшает вероятность отклика по сравнению с A (ATE > 0).
- По CLTV выгода неочевидна: доверительный интервал содержит 0.
- Абсолютная ошибка DR‑оценки относительно истины составляет доли процента, что подтверждает состоятельность метода на синтетике.